In [1]:
%load_ext autoreload
%autoreload 2

# Imports

In [ ]:
import time
from dotenv import load_dotenv
from sklearn.metrics import classification_report, accuracy_score
if not load_dotenv():
    raise Exception("No .env file found")
else:
    print("Environment variables loaded.")
from evaluation import generate_evaluation_dataset, display_confusion_matrix, evaluate_classification_model_on_dataset
from inference import batch_inference, generate_test_dataset

# Configurer le modèle

In [ ]:
valid_models = ["gpt-4o", "gpt-4o-mini"]
while True:
    llm_model = input("Quel modèle voulez-vous utiliser ? Choix : gpt-4o-mini, gpt-4o\n").strip().lower()
    if llm_model in valid_models:
        break
    print("Modèle invalide. Veuillez entrer 'gpt-4o' ou 'gpt-4o-mini'.")
concept = input("Quel critère de classification voulez-vous utiliser ? sujet, sentiment, etc.")
class_1 = input("Quelle est la première classe à considérer ?")
class_2 = input("Quelle est la deuxième classe à considérer ?")

model_parameters = {
    "llm_model": llm_model,
    "concept": concept,
    "class_1": class_1,
    "class_2": class_2
}
print(f"Le modèle utilise les paramètres suivants :\n- LLM : {llm_model}\n- Critère de classification : {concept}\n- Classes : {class_1} | {class_2}")

# Évaluer le modèle

Pour évaluer le modèle, vous pouvez fournir un fichier au format texte (.txt) contenant les verbatim à classifier et leurs classes associées.

Chaque ligne du fichier doit contenir un verbatim suivi de sa classe associée, séparés par " // ".<br>Par exemple :

Quel est le prix de la chaise ? // prix<br>
Je veux payer ma facture. // paiement

In [ ]:
valid_answer = ["oui", "non"]
while True:
    validation_file = input("Avez-vous un fichier de verbatim (.txt) pour évaluer le modèle ? (oui/non)").strip().lower()
    if validation_file in valid_answer:
        break
    print("Réponse invalide. Veuillez entrer 'oui' ou 'non'.")
if validation_file == "oui":
    validation_file = input("Veuillez fournir le chemin du fichier : ")
    with open(validation_file, "r") as f:
        validation_file = f.readlines()
else:
    while True:
        try:
            num_samples = int(input("Pas de souci, nous allons générer un dataset d'évaluation. Combien de verbatim souhaitez-vous générer ? "))
            break
        except ValueError:
            print("Veuillez entrer un nombre entier valide.\n")
    generated_dataset = generate_evaluation_dataset(llm_model=llm_model, concept=concept, class_1=class_1, class_2=class_2, num_samples=num_samples)
    with open(f"datasets/evaluation_dataset_{concept}_{time.strftime("%d_%m_%Y_%H_%M")}.txt", "w") as f:
        f.write(generated_dataset)
    validation_file = generated_dataset.split("\n")
    
print("Aperçu :\n")
for line in validation_file[:5]:
    print(line)

In [ ]:
predictions, ground_truths = evaluate_classification_model_on_dataset(model_parameters, validation_file)

accuracy = accuracy_score(ground_truths, predictions)
print(f"\nAccuracy: {accuracy:.2f}\n")
print(f"Confusion matrix for classification criterion : {concept}")
display_confusion_matrix(ground_truths, predictions, labels=[class_1, class_2])
report = classification_report(ground_truths, predictions, labels=[class_1, class_2])
print(f"F1, recall and precision scores:\n\n{report}")

# Utiliser le modèle

Pour utiliser les prédictions du modèle, vous pouvez fournir un fichier au format texte (.txt) contenant les verbatim à classifier. <br>
Le modèle va prédire la classe de chaque verbatim et expliquer la raison de la prédiction.

In [ ]:
valid_answer = ["oui", "non"]
while True:
    test_file = input("Avez-vous un fichier de verbatim (.txt) pour tester le modèle ? (oui/non)").strip().lower()
    if test_file in valid_answer:
        break
    print("Réponse invalide. Veuillez entrer 'oui' ou 'non'.")

if test_file.lower() == "oui":
    test_file = input("Veuillez fournir le chemin du fichier : ")
    with open(test_file, "r") as f:
        test_file = f.readlines()
else:
    while True:
        try:
            num_samples = int(input("Pas de souci, nous allons générer un dataset de test. Combien de verbatim souhaitez-vous générer ? "))
            break
        except ValueError:
            print("Veuillez entrer un nombre entier valide.\n")
    generated_dataset = generate_test_dataset(llm_model=llm_model, concept=concept, class_1=class_1, class_2=class_2, num_samples=num_samples)
    with open(f"datasets/test_dataset_{concept}_{time.strftime("%d_%m_%Y_%H_%M")}.txt", "w") as f:
        f.write(generated_dataset)
    test_file = generated_dataset.split("\n")
    
print("Aperçu :\n")
for line in test_file[:5]:
    print(line)

In [ ]:
predictions = batch_inference(model_parameters=model_parameters, inputs=test_file)

In [ ]:
for prediction in predictions:
    print(f"Verbatim : {prediction[0]}")
    print(f"- Prédiction : {prediction[1]}")
    print(f"- Raison : {prediction[2]}\n")